
# 2 分词器
参考资料

* [CS366-github仓库](https://github.com/stanford-cs336/assignment1-basics?tab=readme-ov-fil)

* e  https://www.heywhale.com/mw/project/689709e023583639fc675b5c

## 2.1 Unicode 标准
bytes‌：表示原始的8位值（字节序列），每个元素为0-255的整数。主要用于处理二进制数据（如文件、网络传输）。

str（Unicode）‌：表示Unicode字符序列，每个字符由Unicode码点（如U+0041表示字符'A'）定义。用于处理文本数据。

In [ ]:
#
print(ord('s'))

print(ord('牛'))

115
29275


In [ ]:
#Unicode标准中，字符's'的码点是115，字符'牛'的码点是29275
print(chr(115))
print(chr(29275))

s
牛


In [4]:
token = "牛s"
res = [b for b in token.encode("utf-8")]
res

[231, 137, 155, 115]

>### 问题 (1 分) 
>* (a) chr(0)返回什么Unicode字符？
>* (b) 该字符的字符串表示与打印表示有何不同？
>* (c) 当该字符出现在文本中会发生什么？

In [ ]:
chr(0)#'\x00' 该字符不可见

'\x00'

In [5]:
l = "this is a test" + chr(0) + "string"

print(l)

this is a test string


## 2.2 Unicode 编码

In [1]:
test_string ="Hello world 原神牛逼😭"
utf_8_test_string = test_string.encode('utf-8')
print(utf_8_test_string)
print(type(utf_8_test_string))

#拆解字符
print(list(utf_8_test_string))
print("字符长度:", len(test_string), len(utf_8_test_string))


#解码
print(utf_8_test_string.decode('utf-8'))

b'Hello world \xe5\x8e\x9f\xe7\xa5\x9e\xe7\x89\x9b\xe9\x80\xbc\xf0\x9f\x98\xad'
<class 'bytes'>
[72, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100, 32, 229, 142, 159, 231, 165, 158, 231, 137, 155, 233, 128, 188, 240, 159, 152, 173]
字符长度: 17 28
Hello world 原神牛逼😭


> ### 问题 (3 分)
> * (a) 为什么我们更倾向于在UTF-8编码的字节上训练分词器，而不是UTF-16或UTF-32？比较这些编码对不同输入字符串的输出可能有所帮助。
> 
> 交付要求：一至两句话的回答。
> * (b) 考虑以下（错误的）函数，其目的是将UTF-8字节串解码为Unicode字符串。为什么这个函数是错误的？
> ```
>def decode_utf8_bytes_to_str_wrong(bytestring: bytes):  # 注意参数名修正！    
>    return "".join([bytes([b]).decode("utf-8") for b in bytestring])
>```
> 提供一个会产生错误结果的输入字节串示例。


In [ ]:
test_string ="Hello world 牛逼😭"
print(test_string.encode('utf-8'))
print(test_string.encode('utf-16'))
print(test_string.encode('utf-32'))

In [ ]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):  # 注意参数名修正！    
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])

test_string ="Hello world 牛逼😭"
utf_8_test_string = test_string.encode('utf-8')
print(decode_utf8_bytes_to_str_wrong(utf_8_test_string))

#UTF-8特性‌：多字节字符（如中文字符）由多个连续字节组成（如中文字符通常占用3字节）。
# 逐字节解码会将这些字节视为独立字符，导致解码失败（UnicodeDecodeError）。

In [ ]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):  # 注意参数名修正！    
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])

test_string ="Hello world"
utf_8_test_string = test_string.encode('utf-8')
print(decode_utf8_bytes_to_str_wrong(utf_8_test_string))

## 2.3 子字标记化（subword tokenization）
子字标记化（subword tokenization）则介于单词级和字节级之间，提供了一种折中方案。与仅有256个条目的字节级词汇表不同，子词标记器通过扩大词汇量来更高效地压缩原始字节序列。其核心思想是：如果某些字节序列（如 b'the'）在训练数据中频繁出现，就将其合并为一个单独的标记，从而将原本多个字节组成的序列压缩为一个单元。这样既能保持对罕见词和未知词的处理能力，又能显著缩短平均序列长度。

如何选择这些子词单元？Sennrich 等人（2016）提出采用字节对编码（Byte Pair Encoding, BPE），这是一种源自数据压缩技术的算法。BPE 通过迭代地查找并合并出现频率最高的相邻字节对，逐步构建出一组高效的子词单元。每次合并都会引入一个新的符号来代表该字节对，并将其加入词汇表。这一过程持续进行，直到达到预设的词汇表大小。

由于BPE优先合并高频模式，因此最终的词汇表能最大程度地提升整体压缩效率——常见词或词片段更可能被表示为单一标记。在本任务中，我们将实现一种基于字节的BPE分词器，其词汇项由原始字节及其合并后的序列表示。这种方法结合了字节级分词器的鲁棒性与子词级的高效性，在处理未登录词的同时保持合理的序列长度。整个构建词汇表的过程也被称为“训练”BPE分词器，是实现高效文本表示的关键步骤。

## 2.4 BPE 分词器训练
### 2.4.1 词表初始化 
首先进行词汇表的初始化，BPE分词器的词汇表本质上是一个从字节字符串到整数ID的一一映射。由于我们训练的是字节级BPE分词器，初始词汇表包含所有256个可能的字节值，因此初始大小为256。
### 2.4.2 预分词
接下来是预分词阶段。

理论上，我们可以直接在原始字节序列上统计相邻字节对的出现频率并开始合并，但这样每次合并都需要遍历整个语料库，计算成本极高。此外，若不加处理地跨文本合并字节，可能导致仅因标点不同而语义相近的词被拆分为完全不同的一组标记，例如“dog!”和“dog.”会被视为完全无关的标记，不利于模型学习其语义一致性。为缓解这一问题，我们先对语料进行预分词。预分词可以看作是对文本的一次粗粒度切分，有助于高效统计字节对的共现频率。例如，如果单词“text”作为预分词出现了10次，那么其中的字节对‘t’和‘e’的共现次数就可以一次性增加10次，而无需重复扫描整个语料。每个预分词以UTF-8编码的字节序列表示。原始BPE实现中采用简单的空格分割`(s.split(" "))`，而我们则采用GPT-2所使用的正则表达式预分词器（来自OpenAI的tiktoken项目），其模式为：`r"'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"`。例如对句子“`some text that i'll pre-tokenize`”进行切分，结果为：`['some', ' text', ' that', ' i', "'ll", ' pre', '-', 'tokenize']`。在实际编码中，建议使用re.finditer而非re.findall，以便在构建预分词频次映射时避免额外存储中间列表。
### 2.4.3 BPE合并计算
完成预分词后，进入BPE合并的计算阶段。此时，每个预分词已被表示为字节序列，算法开始迭代地统计所有相邻字节对的出现频率，并选择频率最高的字节对进行合并。例如，若字节对（A, B）是当前最频繁的组合，则将所有连续出现的A和B替换为一个新的合并标记“AB”，并将该新标记加入词汇表。由于初始词汇表大小为256，最终词汇表的大小等于256加上训练过程中执行的合并操作数量。

为了提升效率，在统计字节对时不会跨越预分词边界进行合并。当多个字节对频率相同时，采用字典序较大的那一对作为胜出者。例如，若`（"A", "B"）、（"A", "C"）、（"B", "ZZ"）和（"BA", "A"）频率相同，则选择字典序最大的（"BA", "A"）进行合并。此外，某些特殊字符串（如 ）用于表示元信息，例如文档边界或序列结束。这类字符串应作为“特殊标记”被整体保留，绝不允许被拆分。

因此，这些特殊标记必须提前加入词汇表，并分配固定的ID。例如， 表示序列结束，必须始终对应单一标记，以便模型在生成文本时能准确判断何时停止。Sennrich等人（2016）的原始论文中给出了BPE训练的非优化版本，适合初学者实现以理解基本流程。以论文中的示例说明：假设语料包含以下文本：
```
low low low low low    
lower lower widest widest widest    
newest newest newest newest newest newest    
```

并设定`<|endoftext|>`为特殊标记。初始词汇表包括`<|endoftext|>`和全部256个字节。预分词采用空格分割，得到频次统计：low（5次）、lower（2次）、widest（3次）、newest（6次），可表示为字节元组的频数字典，如`{(b'l', b'o', b'w'): 5, ...}`。

随后统计所有相邻字节对的频率：lo（7次）、ow（7次）、we（8次）、er（2次）、wi（3次）、id（3次）、de（3次）、es（9次）、st（9次）、ne（6次）、ew（6次）。其中（es）和（st）频率最高且相同，按字典序选择更大的（st）进行合并。

于是，所有包含“st”的词如“widest”和“newest”中的“s”和“t”被合并为新标记“st”。第二轮中，“e”与“st”组合出现9次，成为最高频对，合并为“est”。继续此过程，后续合并依次为“ow”、“low”、“west”、“ne”等。若仅执行6次合并，最终新增标记为`['s t', 'e st', 'o w', 'l ow', 'w est', 'n e']`。`[<|endoftext|>, [...256 BYTE CHARS], st, est, ow, low, west, ne]`此时，“newest”将被切分为`[ne, west]`两个标记。

这一机制在保持对未知词处理能力的同时，有效压缩了序列长度，提升了模型效率。

In [ ]:
#比较字典序大小
max([("A", "B"), ("A", "C"), ("B", "ZZ"), ("BA", "A")])

## 2.5 BPE 分词器训练实验

>### 问题 （train_bpe_tinystories）：TinyStories 的 BPE 培训（2 分）
>
>* (1) 在 TinyStories 数据集上训练一个字节级 BPE 分词器，最大词汇表大小为 10,000。确保将 TinyStories 的 </s> 特殊标记加入词汇表。将生成的词汇表和合并规则序列化保存到磁盘以便进一步检查。训练耗时多少小时，占用多少内存？词汇表中最长的 token 是什么？这合理吗？
>
>* * 资源要求：≤ 30 分钟（不使用 GPU），≤ 30GB 内存
>
>* * 提示：通过在预分词阶段使用多进程，并结合以下两个事实，你应该能在 2 分钟内完成 BPE 训练：
>* * (a) 数据文件中使用 </s> 标记来分隔文档。
>* * (b) </s> 标记在应用 BPE 合并之前已被作为特殊情况处理。

>交付内容：一到两句话的回复。
>* (2) 对你的代码进行性能分析。分词器训练过程中哪一部分耗时最长？

>交付内容：一到两句话的回复。

In [2]:
%run "2.5 BPE 分词器训练.py"

🚀 开始训练
📖 从 ./data/TinyStoriesV2-GPT4-train.txt 加载并采样 22000 个文档...
预分词调用线程数: 15


预分词: 100%|██████████| 22000/22000 [00:10<00:00, 2158.53it/s]


✅ 预分词完成，得到 4,308,884 个token序列
🔧 构建BPE索引...
🔄 开始BPE训练，目标合并数: 9,741


训练BPE: 100%|██████████| 9741/9741 [1:55:02<00:00,  1.41合并/s]   



✅ 训练完成! 耗时: 6955.33秒

🔬 小规模验证
📖 从 ./data/TinyStoriesV2-GPT4-valid.txt 加载并采样 500 个文档...
预分词调用线程数: 15


预分词: 100%|██████████| 500/500 [00:00<00:00, 5192.09it/s]


✅ 预分词完成，得到 97,365 个token序列
🔧 构建BPE索引...
🔄 开始BPE训练，目标合并数: 9,741


训练BPE:  59%|█████▉    | 5761/9741 [00:33<00:23, 169.62合并/s]


⚠️ 没有更多有效的字符对可供合并，提前结束训练

📊 训练结果
训练词汇表大小: 10,000
训练合并操作数: 9,741
验证词汇表大小: 6,020
验证合并操作数: 5,761

📈 词汇表重叠率: 57.3%

🔍 分词器评估
样例文本: u don't have to be scared of the loud dog, I'll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him ...
词汇表大小: 10,000
唯一token数: 10,000
重复 token 数: 0
token 字节长度: min=1, avg=5.75, max=14
合并操作数: 9,741

示例 token（按 id 升序，最多 20）：
 0: hex=00  decoded= 
 1: hex=01  decoded=
 2: hex=02  decoded=
 3: hex=03  decoded=
 4: hex=04  decoded=
 5: hex=05  decoded=
 6: hex=06  decoded=
 7: hex=07  decoded=
 8: hex=08  decoded
 9: hex=09  decoded=	
 10: hex=0a  decoded=

 11: hex=0b  decoded=
 12: hex=0c  decoded=
 13: hex=0d  decoded=
 14: hex=0e  decoded=
 15: hex=0f  decoded=
 16: hex=10  decoded=
 17: hex=11  decoded=
 18: hex=12  decoded=
 19: hex=13  decoded=
✅ 词汇表已保存至: ./out\gpt2_vocab.json
✅ 合并列表已保存至: ./out\gpt2_merges.txt
💾 峰值内存使用: 1.82 GB


## 2.6 BPE分词器编码与解码
假设根据上述训练，我们获得了**词汇表**`vocab` 和 **合并列表** `merges`。

现在我们要对Unicode字符 `'the cat ate'` 进行编码。 


In [ ]:
#词汇表<int,bytes>
vocab={    
  0: b' ',     # 空格    
  1: b'a',    
  2: b'c',    
  3: b'e',    
  4: b'h',    
  5: b't',    
  6: b'th',    
  7: b' c',   # 空格 + c    
  8: b' a',   # 空格 + a    
  9: b'the',    
 10: b' at'   # 空格 + a + t    
 #......
}

#合并规则
merges=[    
  (b't', b'h'),      # → b'th'    
  (b' ', b'c'),      # → b' c'    
  (b' ', b'a'),      # → b' a'    
  (b'th', b'e'),     # → b'the'    
  (b' a', b't')      # → b' at'    
  #......
]

预分词结果`['the', ' cat', ' ate']`  

例如 预分词`b'the'`

初始分解：`[b't', b'h', b'e']`

应用合并：
* `(b't', b'h')` → 合并为 `[b'th', b'e']`   
* `(b'th', b'e')` → 合并为 `[b'the']`

查表得 ID：`b'the'` → `ID 9`

ID为 `[9]` 其他同理

`b' cat'` ID 为`[7,1,5]`



将各预分词的编码结果串联：

*    'the' → [9]
*    ' cat' → [7, 1, 5]
*    ' ate' → [10, 3]

最终编码结果：
* [9, 7, 1, 5, 10, 3]

解码非常简单，就是上述操作的逆


In [3]:
%run "2.6 BPE 分词器编码.py"

🚀 加载训练好的分词器...
✅ 分词器加载成功!

🔍 开始分词器测试...

文本: Wow, that is great
编码 (9 tokens): [1631, 44, 32, 258, 32, 291, 32, 3771, 301]
解码: Wow, <unk> is great
⚠️ 往返不一致
原始: Wow, that is great
解码: Wow, <unk> is great
位置 5: 原始 't' (U+0074) vs 解码 '<' (U+003C)
Token示例: ['Wow', ',', ' ', '<unk>', ' ', 'is', ' ', 'gre', 'at']

文本: you can eat
编码 (6 tokens): [9055, 32, 4591, 32, 370, 116]
解码: you can eat
✅ 往返一致
Token示例: ['you', ' ', 'can', ' ', 'ea', 't']

文本: This is a test with special tokens: <|endoftext|>
编码 (24 tokens): [2038, 32, 291, 32, 97, 32, 116, 2441, 32, 119, 2951, 32, 1866, 528, 105, 350, 32, 624, 107, 306]...
解码: This is a test with special tokens: <|endoftext|>
✅ 往返一致
Token示例: ['This', ' ', 'is', ' ', 'a', ' ', 't', 'est', ' ', 'w']

文本: Special token test: <|endoftext|>
编码 (14 tokens): [2176, 528, 105, 350, 32, 624, 107, 306, 32, 116, 2441, 58, 32, 256]
解码: Special token test: <|endoftext|>
✅ 往返一致
Token示例: ['Sp', 'ec', 'i', 'al', ' ', 'to', 'k', 'en', ' ', 't']

✅ 测试完成!
